# Des mots aux sacs de mots

La tokenisation consiste à découper un texte en *token*, l'approche *sac de mots* consiste à compter les occurences de chaque mot dans chaque document de la base de données.

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [2]:
texte = """
Mardi 20 février, à la médiathèque des Mureaux (Yvelines), le chef de l’Etat a accompagné 
la locataire de la rue de Valois pour la remise officielle du rapport 
sur les bibliothèques, rédigé par leur ami commun, l’académicien 
Erik Orsenna, avec le concours de Noël Corbin, inspecteur général 
des affaires culturelles. L’occasion de présenter les premières 
mesures en faveur d’un « plan bibliothèques ».
"""

## Pipeline de traitement

Maintenant qu'on sait découper en mots ou couples de mots, il faut appliquer sur une liste de textes. On crée une petite liste de textes.

In [3]:
import pandas
df = pandas.DataFrame(dict(text=[texte, "tout petit texte"]))
df

,text
0,"\nMardi 20 février, à la médiathèque des Murea..."
1,tout petit texte


Et on applique l'objet [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer) :

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
cd = CountVectorizer()
cd.fit(df["text"])
res = cd.transform(df["text"])
res

<2x51 sparse matrix of type '<class 'numpy.int64'>'
	with 51 stored elements in Compressed Sparse Row format>

On récupère une [matrice sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html) où chaque colonne compte le nombre d'occurence d'un mot dans le texte :

In [5]:
res.todense()

matrix([[1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 5, 2, 1, 1, 1, 1, 1, 1, 1,
         1, 4, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 1, 1, 0, 0, 0]], dtype=int64)

Les mots sont les suivants :

In [6]:
cd.vocabulary_

{'20': 0,
 'académicien': 1,
 'accompagné': 2,
 'affaires': 3,
 'ami': 4,
 'avec': 5,
 'bibliothèques': 6,
 'chef': 7,
 'commun': 8,
 'concours': 9,
 'corbin': 10,
 'culturelles': 11,
 'de': 12,
 'des': 13,
 'du': 14,
 'en': 15,
 'erik': 16,
 'etat': 17,
 'faveur': 18,
 'février': 19,
 'général': 20,
 'inspecteur': 21,
 'la': 22,
 'le': 23,
 'les': 24,
 'leur': 25,
 'locataire': 26,
 'mardi': 27,
 'mesures': 28,
 'mureaux': 29,
 'médiathèque': 30,
 'noël': 31,
 'occasion': 32,
 'officielle': 33,
 'orsenna': 34,
 'par': 35,
 'petit': 36,
 'plan': 37,
 'pour': 38,
 'premières': 39,
 'présenter': 40,
 'rapport': 41,
 'remise': 42,
 'rue': 43,
 'rédigé': 44,
 'sur': 45,
 'texte': 46,
 'tout': 47,
 'un': 48,
 'valois': 49,
 'yvelines': 50}

## Un tokenizer différent

La classe [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer) est facilement paramétrable. On peut en particulier changer le *tokenizer* :

In [7]:
from nltk.tokenize import word_tokenize

In [8]:
count_vect = CountVectorizer(tokenizer=word_tokenize)
counts = count_vect.fit_transform(df["text"])
counts.shape

(2, 62)

In [9]:
counts.todense()

matrix([[1, 1, 6, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 5, 2, 1,
         1, 1, 1, 1, 1, 1, 1, 3, 4, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 4],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int64)

## Hashing

Le nombre de mots distincts peut être très grand surtout si la source des textes est bruitée (faute d'orthographe, spams, ...). Pour réduire le nombre de mots, on peut utiliser un *hash* à valeur dans un ensemble plus petit que le nombre de mots découverts : c'est une sorte de modulo. Deux mots pourront être comptabilisés dans la même colonne. On utilise la classe [HashingVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html#sklearn.feature_extraction.text.HashingVectorizer).

In [10]:
from sklearn.feature_extraction.text import HashingVectorizer
cd = HashingVectorizer(n_features=5)
cd.fit(df["text"])
res = cd.transform(df["text"])
res.todense()

matrix([[-0.08247861,  0.16495722, -0.41239305,  0.74230749,  0.49487166],
        [-0.4472136 ,  0.        ,  0.89442719,  0.        ,  0.        ]])

La classe utilise la classe [FeatureHasher](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.FeatureHasher.html) et plus précisément le code dans [_hashing.pyx](https://github.com/scikit-learn/scikit-learn/blob/a24c8b464d094d2c468a16ea9f8bf8d42d949f84/sklearn/feature_extraction/_hashing.pyx).

In [11]:
cd = HashingVectorizer(n_features=5, binary=True)
cd.fit(df["text"])
res = cd.transform(df["text"])
res.todense()

matrix([[0.4472136 , 0.4472136 , 0.4472136 , 0.4472136 , 0.4472136 ],
        [0.70710678, 0.        , 0.70710678, 0.        , 0.        ]])

Réduire les dimensions tout en gardant une certaine forme de proximité.

In [12]:
df2 = pandas.DataFrame(dict(text=[texte, 
                                  " ".join(texte.split()[1:-1]), 
                                  " ".join(texte.split()[5:-5]), 
                                  " ".join(texte.split()[10:-10]) + ' machine', 
                                  " ".join(texte.split()[20:-20]) + ' learning', 
                                  " ".join(texte.split()[25:-25]) + ' statistique', 
                                  " ".join(texte.split()[30:-30]) + ' nouveau', 
                                  "tout petit texte"]))
df2

,text
0,"\nMardi 20 février, à la médiathèque des Murea..."
1,"20 février, à la médiathèque des Mureaux (Yvel..."
2,"médiathèque des Mureaux (Yvelines), le chef de..."
3,chef de l’Etat a accompagné la locataire de la...
4,de Valois pour la remise officielle du rapport...
5,"officielle du rapport sur les bibliothèques, r..."
6,"bibliothèques, rédigé par nouveau"
7,tout petit texte


In [13]:
cd = HashingVectorizer(n_features=8, binary=False)
cd.fit(df2["text"])
res = cd.transform(df2["text"])
res.todense()

matrix([[-0.08873565,  0.3549426 , -0.1774713 ,  0.        , -0.26620695,
          0.        ,  0.79862086,  0.3549426 ],
        [-0.09053575,  0.36214298, -0.18107149,  0.        , -0.18107149,
          0.        ,  0.81482171,  0.36214298],
        [-0.10783277,  0.43133109, -0.21566555,  0.        ,  0.        ,
          0.        ,  0.75482941,  0.43133109],
        [-0.24806947,  0.3721042 ,  0.        , -0.12403473, -0.12403473,
          0.        ,  0.86824314,  0.12403473],
        [-0.20412415,  0.81649658,  0.        ,  0.20412415,  0.20412415,
          0.20412415,  0.40824829,  0.        ],
        [ 0.        ,  0.35355339,  0.35355339,  0.35355339,  0.        ,
          0.70710678, -0.35355339,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.70710678, -0.70710678],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  1.        ,  0.        ]])

In [14]:
from sklearn.metrics.pairwise import pairwise_distances
pandas.DataFrame(pairwise_distances(res))

,0,1,2,3,4,5,6,7
0,0.000000,0.087352,0.293731,0.388511,0.916931,1.561800,1.171556,0.634632
1,0.087352,0.000000,0.217844,0.368633,0.883337,1.564650,1.166111,0.608569
2,0.293731,0.217844,0.000000,0.455795,0.797058,1.543129,1.241976,0.700244
3,0.388511,0.368633,0.455795,0.000000,0.826704,1.561579,0.973412,0.513336
4,0.916931,0.883337,0.797058,0.826704,0.000000,1.130625,1.192749,1.087889
5,1.561800,1.564650,1.543129,1.561579,1.130625,0.000000,1.581139,1.645329
6,1.171556,1.166111,1.241976,0.973412,1.192749,1.581139,0.000000,0.765367
7,0.634632,0.608569,0.700244,0.513336,1.087889,1.645329,0.765367,0.000000


## tf-idf

Ce genre de technique produit des matrices de très grande dimension qu'il faut réduire. On peut enlever les mots rares ou les mots très fréquents. [td-idf](https://fr.wikipedia.org/wiki/TF-IDF>) est une technique qui vient des moteurs de recherche. Elle construit le même type de matrice (même dimension) mais associe à chaque couple (document - mot) un poids qui dépend de la fréquence d'un mot globalement et du nombre de documents contenant ce mot.

$$idf(t) = \log \frac{\# D}{\#\{d \; | \; t \in d \}}$$

Où :

- $\#D$ est le nombre de documents
- $\#\{d \; | \; t \in d \}$ est le nombre de documents contenant le mot $t$

$f(t,d)$ est le nombre d'occurences d'un mot $t$ dans un document $d$.

$$tf(t,d) = \frac{1}{2} + \frac{1}{2} \frac{f(t,d)}{\max_{t' \in d} f(t',d)}$$

On construit le nombre $tfidf(t,f) = tf(t,d) idf(t)$ :

Le terme $idf(t)$ favorise les mots présent dans peu de documents, le terme $tf(t,f)$ favorise les termes répétés un grand nombre de fois dans le même document. On applique à la matrice précédente. Sur deux documents, cela ne sert pas à grand-chose. On utilise la classe [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer).

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
cd = TfidfVectorizer()
cd.fit(df["text"])
res = cd.transform(df["text"])
res.todense()

matrix([[0.10050378, 0.10050378, 0.10050378, 0.10050378, 0.10050378,
         0.10050378, 0.20100756, 0.10050378, 0.10050378, 0.10050378,
         0.10050378, 0.10050378, 0.50251891, 0.20100756, 0.10050378,
         0.10050378, 0.10050378, 0.10050378, 0.10050378, 0.10050378,
         0.10050378, 0.10050378, 0.40201513, 0.20100756, 0.20100756,
         0.10050378, 0.10050378, 0.10050378, 0.10050378, 0.10050378,
         0.10050378, 0.10050378, 0.10050378, 0.10050378, 0.10050378,
         0.10050378, 0.        , 0.10050378, 0.10050378, 0.10050378,
         0.10050378, 0.10050378, 0.10050378, 0.10050378, 0.10050378,
         0.10050378, 0.        , 0.        , 0.10050378, 0.10050378,
         0.10050378],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0. 

Si on décompose 

In [16]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(CountVectorizer(), TfidfTransformer())
pipe.fit(df['text'])
res = pipe.transform(df['text'])
res.todense()

matrix([[0.10050378, 0.10050378, 0.10050378, 0.10050378, 0.10050378,
         0.10050378, 0.20100756, 0.10050378, 0.10050378, 0.10050378,
         0.10050378, 0.10050378, 0.50251891, 0.20100756, 0.10050378,
         0.10050378, 0.10050378, 0.10050378, 0.10050378, 0.10050378,
         0.10050378, 0.10050378, 0.40201513, 0.20100756, 0.20100756,
         0.10050378, 0.10050378, 0.10050378, 0.10050378, 0.10050378,
         0.10050378, 0.10050378, 0.10050378, 0.10050378, 0.10050378,
         0.10050378, 0.        , 0.10050378, 0.10050378, 0.10050378,
         0.10050378, 0.10050378, 0.10050378, 0.10050378, 0.10050378,
         0.10050378, 0.        , 0.        , 0.10050378, 0.10050378,
         0.10050378],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0. 

Ca marche aussi sur les hash.

In [17]:
pipe = make_pipeline(HashingVectorizer(n_features=5), TfidfTransformer())
pipe.fit(df['text'])
res = pipe.transform(df['text'])
res.todense()

matrix([[-0.06142775,  0.17266912, -0.30713875,  0.77701104,  0.51800736],
        [-0.4472136 ,  0.        ,  0.89442719,  0.        ,  0.        ]])

## Sur un jeu de données

L'idée est d'appliquer une LDA ou [Latent Dirichet Application](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html).

In [18]:
from papierstat.datasets import load_tweet_dataset
tweet = load_tweet_dataset()
tweet = tweet[tweet["text"].notnull()]
tweet.head(n=2).T

,0,1
index,776066992054861825,776067660979245056
nb_user_mentions,0,0
nb_extended_entities,0,0
nb_hashtags,1,1
geo,NaN,NaN
text_hashtags,", SiJétaisPrésident",", SiJétaisPrésident"
annee,2016,2016
delimit_mention,NaN,NaN
lang,fr,fr
id_str,7.76067e+17,7.76068e+17


In [19]:
pipeline = make_pipeline(CountVectorizer(), TfidfTransformer())
res = pipeline.fit_transform(tweet['text'])

In [20]:
count = pipeline.steps[0][-1]
voc = count.get_feature_names()

Le code suivant marche parce que la base n'est pas trop petite.

In [21]:
data = pandas.DataFrame(res.todense(), columns=voc)
data.head()

,00,000,0000,0079,00h,04,06,09,0ccnpoxuwu,0cxdedblpx,...,île,îles,îls,œil,œufs,œuvre,œuvrer,œuvrerais,œuvres,δlex
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10)
lda.fit(res)

c:\Python364_x64\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [23]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [24]:
print_top_words(lda, voc, 10)

Topic #0:
sijetaispresident je de les le la et en co https
Topic #1:
interdirais port camembert pizza leggings ballerine légaliserai rendrai sijetaispresident léopard
Topic #2:
revenu dormirais affaires concert bercy commun universel hiver lelab_e1 grande
Topic #3:
sijetaispresident la de je les le et co https des
Topic #4:
démissionnerais volonté mets ss10 organiserais officielle soirées urgent cache offrirais
Topic #5:
maths lait mandat abolirai vive céréales mettent allait propose morsay
Topic #6:
société trahison haute légalisation marché michel tahaetmourad massive exercer keen
Topic #7:
supprimerais obligatoire lundi rendrais sieste vendredi bac promesses férié imposerai
Topic #8:
bah les dictateur sijetaispresident justice devoirs instaurerais arrêterai nationale police
Topic #9:
sijetaispresident mcdo national hymne le chocolatine co https écoles cannabis



In [25]:
comp = lda.transform(res)

In [26]:
comp.shape

(5087, 10)

Pour chaque tweet, le score plus élevé indique la classe dans laquelle classé le tweet.

In [27]:
comp[0,:]

array([0.02704576, 0.02703572, 0.02703569, 0.75666716, 0.02703569,
       0.02703569, 0.02703569, 0.02703571, 0.02703572, 0.02703717])